# 「する」の頻出度合いを調査

学習時に「する」というfeatureが相当高頻度であることが確認されました。

状況を見るために 'test_benefitone_conversation.csv' だけ使用して調査しています。

## (1) テストデータ／環境準備

マイオペで使用しているテストデータ（learning/tests/engine/fixtures/ 配下のCSVファイル）をベースに動作確認を行います。

動作確認にあたっては、MySQLdb に接続できないため、ローカル環境テスト用の Bot クラスを使用しています。

In [1]:
'''
    テスト環境を準備するためのモジュールを使用します。
'''
import sys
import os
learning_dir = os.path.abspath("../../") #<--- donusagi-bot/learning
os.chdir(learning_dir)

if learning_dir not in sys.path:
    sys.path.append(learning_dir)

from prototype.modules import TestTool

In [2]:
'''
    データファイルは、既存の訓練データを別場所にコピーしてから使用します
    テストデータは、csv_file_name で指定した複数件のファイルを使用します。
'''
csv_file_names = [
    'test_benefitone_conversation.csv',
]
copied_csv_file_paths = TestTool.copy_testdata_csv(learning_dir, csv_file_names)
copied_csv_file_paths

CSV file for test=[/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv]


['/Users/makmorit/GitHub/donusagi-bot/learning/prototype/resources/test_benefitone_conversation.csv']

## (2) TF-IDFベクターの準備

Bot クラス内に組み込まれている __build_training_set_from_csv 関数をバラして実行しています。

In [3]:
'''
    初期設定
    データファイル、エンコードを指定
    内容は、learn.py を参考にしました。    
'''
from learning.core.learn.learning_parameter import LearningParameter
attr = {
    'include_failed_data': False,
    'include_tag_vector': False,
    'classify_threshold': None,
    # 'algorithm': LearningParameter.ALGORITHM_NAIVE_BAYES
    'algorithm': LearningParameter.ALGORITHM_LOGISTIC_REGRESSION,
    # 'params_for_algorithm': { 'C': 200 }
    'params_for_algorithm': {}
}
learning_parameter = LearningParameter(attr)

bot_id = 7777
csv_file_encoding = 'utf-8'

### (2-1) 訓練データのTF-IDFベクター

In [4]:
'''
    訓練データの生成（内部で TF-IDF 処理を実行）

    TrainingMessageFromCsv は、
    ファイル名のリスト（copied_csv_file_paths）を受け取ると、
    Dataset を連結してくれるようカスタマイズしてあります。
'''
#from learning.core.training_set.training_message_from_csv import TrainingMessageFromCsv
from prototype.modules.training_message_from_csv import TrainingMessageFromCsv
training_set = TrainingMessageFromCsv(bot_id, copied_csv_file_paths, learning_parameter, encoding=csv_file_encoding)
build_training_set_from_csv = training_set.build()

X = build_training_set_from_csv.x
y = build_training_set_from_csv.y

2017/03/22 PM 12:02:05 TrainingMessageFromCsv#__build_learning_training_messages count of learning data: 4114
2017/03/22 PM 12:02:05 TextArray#__init__ start
2017/03/22 PM 12:02:06 TextArray#to_vec start
2017/03/22 PM 12:02:06 TextArray#to_vec end


In [5]:
n_sample = X.shape[0]
n_feature = X.shape[1]
print("sample=%d, feature=%d" % (n_sample, n_feature))

sample=4114, feature=646


### (2-2) 単語の頻出度合いをチェック

In [6]:
from learning.core.training_set.text_array import TextArray
text_array = build_training_set_from_csv._body_array
vocabulary = text_array._vectorizer.vocabulary_

#### featureが抽出されたサンプルの件数を、featureごとに集計します。

In [9]:
ext_feature_count = [0.0 for i in range(n_feature)]
tot_feature_count = 0.0
for i in range(n_sample):
    
    arr = X[i].toarray()[0]
    for j in range(n_feature):
        if arr[j] == 0.0:
            continue
        ext_feature_count[j] += 1.0
        tot_feature_count += 1.0

#### 降順ソートして表示（上位５０件）

「する」が群を抜いて上位にランキングされております。

In [12]:
ext_feature_list = []
for k, v in enumerate(ext_feature_count):
    item = TestTool.get_item_from_vocabulary(vocabulary, k)
    elem = (item, v, v / tot_feature_count)
    ext_feature_list.append(elem)

sorted_list = sorted(ext_feature_list, key=lambda x: x[1], reverse=True)

for i in range(50):
    print("[%s] count=%d, rate=%0.4f" % sorted_list[i])

[する] count=842, rate=0.0609
[できる] count=338, rate=0.0244
[借りる] count=226, rate=0.0163
[欲しい] count=222, rate=0.0160
[使う] count=164, rate=0.0119
[使える] count=162, rate=0.0117
[携帯] count=140, rate=0.0101
[カード] count=126, rate=0.0091
[使用] count=120, rate=0.0087
[会議] count=118, rate=0.0085
[iphone] count=112, rate=0.0081
[便] count=112, rate=0.0081
[利用] count=108, rate=0.0078
[ない] count=104, rate=0.0075
[コード] count=104, rate=0.0075
[ファイル] count=104, rate=0.0075
[室] count=104, rate=0.0075
[れる] count=100, rate=0.0072
[社内] count=98, rate=0.0071
[交換] count=96, rate=0.0069
[希望] count=96, rate=0.0069
[可能] count=94, rate=0.0068
[いただける] count=92, rate=0.0067
[機] count=92, rate=0.0067
[教える] count=90, rate=0.0065
[プリンター] count=86, rate=0.0062
[いい] count=84, rate=0.0061
[もらえる] count=84, rate=0.0061
[レンタカー] count=84, rate=0.0061
[壊れる] count=82, rate=0.0059
[コーヒー] count=78, rate=0.0056
[テープ] count=78, rate=0.0056
[無い] count=78, rate=0.0056
[メッセージ] count=76, rate=0.0055
[変更] count=76, rate=0.0055
[封筒] coun